# DX 704 Week 2 Project

This week's project will analyze fresh strawberry price data for a hypothetical "buy low, freeze, and sell high" business.
Strawberries show strong seasonality in their prices compared to other fruits.

![](https://ers.usda.gov/sites/default/files/_laserfiche/Charts/61401/oct14_finding_plattner_fig01.png)

Image source: https://www.ers.usda.gov/amber-waves/2014/october/seasonal-fresh-fruit-price-patterns-differ-across-commodities-the-case-of-strawberries-and-apples

You are considering a business where you buy strawberries when the prices are very low, carefully freeze them, even more carefully defrost them, and then sell them when the prices are high.
You will forecast strawberry price time series and then use them to tactically pick times to buy, freeze, and sell the strawberries.

The full project description, a template notebook, and raw data are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-02


### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Backtest Strawberry Prices

Read the provided "strawberry-prices.tsv" with data from 2000 through 2024.
This data is based on data from the U.S. Bureau of Statistics, but transformed so the ground truth is not online.
https://fred.stlouisfed.org/series/APU0000711415

Use the data for 2000 through 2023 to predict monthly prices in 2024.
Spend some time to make sure you are happy with your methodology and prediction accuracy, since you will reuse the methodology to forecast 2025 next.
Save the 2024 backtest predictions as "strawberry-backtest.tsv" with columns month and price.


In [1]:

import pandas as pd
from pathlib import Path

# 1) Load
df = pd.read_csv("strawberry-prices.tsv", sep="\t").rename(
    columns=lambda c: c.lower()
)
df["month"] = pd.to_datetime(df["month"])
df = df.sort_values("month").reset_index(drop=True)

# 2) Split
train = df[df["month"].dt.year <= 2023].copy()
ref2023 = train[train["month"].dt.year == 2023].copy()
ref2023["m"] = ref2023["month"].dt.month

# 3) Fallback seasonal means
seasonal_mean = (
    train.assign(m=train["month"].dt.month)
         .groupby("m")["price"].mean()
)

# 4) Building 2024 predictions by same-month-as-2023 (fallback to monthly mean)
months_2024 = pd.date_range("2024-01-01", "2024-12-01", freq="MS")
pred_rows = []
for dt in months_2024:
    m = dt.month
    last = ref2023.loc[ref2023["m"] == m, "price"]
    yhat = float(last.iloc[0]) if not last.empty else float(seasonal_mean.loc[m])
    pred_rows.append({"month": dt.strftime("%Y-%m"), "price": yhat})

backtest = pd.DataFrame(pred_rows, columns=["month", "price"])

# 5) Saving as TSV
out_path = Path("strawberry-backtest.tsv")
backtest.to_csv(out_path, sep="\t", index=False)

Submit "strawberry-backtest.tsv" in Gradescope.

## Part 2: Backtest Errors

What are the mean and standard deviation of the residuals between your backtest predictions and the ground truth? (If your mean is not close to zero, then you may be missing a long term trend.)

Write the mean and standard deviation to a file "backtest-accuracy.tsv" with two columns, mean and std.

In [4]:
import pandas as pd

truth = (
    pd.read_csv("strawberry-prices.tsv", sep="\t")
      .rename(columns=str.lower)
)
truth["month"] = pd.to_datetime(truth["month"])
truth_2024 = truth[truth["month"].dt.year == 2024].copy()
truth_2024["month"] = truth_2024["month"].dt.strftime("%Y-%m")

pred = pd.read_csv("strawberry-backtest.tsv", sep="\t")

df = truth_2024.merge(pred, on="month", suffixes=("_true","_pred"))
df["residual"] = df["price_true"] - df["price_pred"]  # actual - predicted

mean_res = df["residual"].mean()
std_res  = df["residual"].std(ddof=1)

out = pd.DataFrame({"mean":[mean_res], "std":[std_res]})
out.to_csv("backtest-accuracy.tsv", sep="\t", index=False)

df["price_pred_adj"] = df["price_pred"] + mean_res

Submit "backtest-accuracy.tsv" in Gradescope.

## Part 3: Forecast Strawberry Prices

Use all the data from 2000 through 2024 to predict monthly prices in 2025 using the same methodology from part 1.
Make a monthly forecast for each month of 2025 and save it as "strawberry-forecast.tsv" with columns for month and price.

In [5]:
import pandas as pd

# 1) Loading and preping
df = pd.read_csv("strawberry-prices.tsv", sep="\t").rename(columns=str.lower)
df["month"] = pd.to_datetime(df["month"])
df = df.sort_values("month").reset_index(drop=True)

# 2) Grabing 2024 as the seasonal reference
ref = df[df["month"].dt.year == 2024].copy()
ref["m"] = ref["month"].dt.month

# 3) Fallback: monthly mean over 2000–2024 in case any 2024 month is missing
seasonal_mean = (
    df[df["month"].dt.year <= 2024]
      .assign(m=lambda d: d["month"].dt.month)
      .groupby("m")["price"].mean()
)

# 4) Building 2025 forecast
months_2025 = pd.date_range("2025-01-01", "2025-12-01", freq="MS")
rows = []
for dt in months_2025:
    m = dt.month
    same_month_2024 = ref.loc[ref["m"] == m, "price"]
    yhat = float(same_month_2024.iloc[0]) if not same_month_2024.empty else float(seasonal_mean.loc[m])
    rows.append({"month": dt.strftime("%Y-%m"), "price": yhat})

forecast = pd.DataFrame(rows, columns=["month", "price"])

# 5) Save
forecast.to_csv("strawberry-forecast.tsv", sep="\t", index=False)

Submit "strawberry-forecast.tsv" in Gradescope.

## Part 4: Buy Low, Freeze and Sell High

Using your 2025 forecast, analyze the profit picking different pairs of months to buy and sell strawberries.
Maximize your profit assuming that it costs &dollar;0.20 per pint to freeze the strawberries, &dollar;0.10 per pint per month to store the frozen strawberries and there is a 10% price discount from selling previously frozen strawberries.
So, if you buy a pint of strawberies for &dollar;1, freeze them, and sell them for &dollar;2 three months after buying them, then the profit is &dollar;2 * 0.9 - &dollar;1 - &dollar;0.20 - &dollar;0.10 * 3 = &dollar;0.30 per pint.
To evaluate a given pair of months, assume that you can invest &dollar;1,000,000 to cover all costs, and that you buy as many pints of strawberries as possible.

Write the results of your analysis to a file "timings.tsv" with columns for the buy_month, sell_month, pints_purchased, and expected_profit.

In [6]:
# parameters
BUDGET = 1_000_000
FREEZE = 0.20
STORAGE = 0.10
DISCOUNT = 0.10  # the 10% off sell price

# loading 2025 forecast
F = pd.read_csv("strawberry-forecast.tsv", sep="\t")
F["month"] = pd.to_datetime(F["month"])
prices = F.set_index(F["month"].dt.month)["price"].to_dict()

rows = []
for buy_m in range(1, 13):
    for sell_m in range(buy_m + 1, 13):
        months_held = sell_m - buy_m
        buy_price = prices[buy_m]
        sell_price = prices[sell_m]

        per_pint_cost = buy_price + FREEZE + STORAGE * months_held
        per_pint_profit = (sell_price * (1 - DISCOUNT)) - per_pint_cost

        pints = int(BUDGET // per_pint_cost) if per_pint_cost > 0 else 0
        total_profit = per_pint_profit * pints

        rows.append({
            "buy_month": f"2025-{buy_m:02d}",
            "sell_month": f"2025-{sell_m:02d}",
            "pints_purchased": pints,
            "expected_profit": total_profit
        })

timings = pd.DataFrame(rows).sort_values("expected_profit", ascending=False)
timings.to_csv("timings.tsv", sep="\t", index=False)

Submit "timings.tsv" in Gradescope.

## Part 5: Strategy Check

What is the best profit scenario according to your previous timing analysis?
How much does that profit change if the sell price is off by one standard deviation from your backtest analysis?
(Variation in the sell price is more dangerous because you can see the buy price before fully committing.)

Write the results to a file "check.tsv" with columns best_profit and one_std_profit.

In [7]:
# best profit vs. sell-price minus 1 std

# parameters
BUDGET = 1_000_000
FREEZE = 0.20
STORAGE = 0.10
DISCOUNT = 0.10  # the 10% off sell price

# 1) Loading inputs
timings = pd.read_csv("timings.tsv", sep="\t")
forecast = pd.read_csv("strawberry-forecast.tsv", sep="\t")
acc = pd.read_csv("backtest-accuracy.tsv", sep="\t")
sell_std = float(acc.loc[0, "std"])

# 2) Pick best scenario from timings
best = timings.loc[timings["expected_profit"].idxmax()].to_dict()
buy_m  = int(str(best["buy_month"]).split("-")[1])
sell_m = int(str(best["sell_month"]).split("-")[1])

# 3) Get prices for those months
forecast["month_num"] = forecast["month"].str.slice(5,7).astype(int)
buy_price  = float(forecast.loc[forecast["month_num"]==buy_m, "price"].iloc[0])
sell_price = float(forecast.loc[forecast["month_num"]==sell_m, "price"].iloc[0])

months_held = sell_m - buy_m
per_pint_cost = buy_price + FREEZE + STORAGE * months_held
pints = int(BUDGET // per_pint_cost)

# 4) Baseline best profit (should match timings)
per_pint_profit = (sell_price * (1 - DISCOUNT)) - per_pint_cost
best_profit = per_pint_profit * pints

# 5) One-std worse sell price (downward shock)
sell_price_worst = sell_price - sell_std
per_pint_profit_worst = (sell_price_worst * (1 - DISCOUNT)) - per_pint_cost
one_std_profit = per_pint_profit_worst * pints

# 6) Save results
out = pd.DataFrame({"best_profit":[best_profit], "one_std_profit":[one_std_profit]})
out.to_csv("check.tsv", sep="\t", index=False)

Submit "check.tsv" in Gradescope.

## Part 6: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


In [11]:
ack_text = """Acknowledgments

I used ChatGPT as a study partner to clarify  and check errors, and simplify code while completing this project. 


For transparency, I am including links to the transcripts of these interactions:

1. ChatGPT guidance: [https://chatgpt.com/share/68bbb59b-63d0-8007-a938-1be080ca31e3]

These resources document the support I received and ensure compliance with the class generative AI policy.
"""

with open("acknowledgments.txt","w") as f:
    f.write(ack_text)


Submit "acknowledgements.txt" in Gradescope.

## Part 7: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.